In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random as r
from datetime import datetime
from scipy import stats

In [2]:
import sqlite3 as sl#connecct to database object
conn = sl.connect('Elections.db')
c = conn.cursor()

In [3]:
clinton = pd.read_sql('''select * from fivethirtyeight''',conn)
clinton

,index,cycle,branch,type,matchup,forecastdate,state,startdate,enddate,pollster,...,adjpoll_clinton,adjpoll_trump,adjpoll_johnson,adjpoll_mcmullin,multiversions,url,poll_id,question_id,createddate,timestamp
0,0,2016,President,polls-plus,Clinton vs. Trump vs. Johnson,11/1/16,U.S.,10/25/2016,10/31/2016,Google Consumer Surveys,...,42.64140,40.86509,5.675099,NaN,None,https://datastudio.google.com/u/0/#/org//repor...,47940,74999,11/1/16,15:09:38 1 Nov 2016
1,1,2016,President,polls-plus,Clinton vs. Trump vs. Johnson,11/1/16,U.S.,10/27/2016,10/30/2016,ABC News/Washington Post,...,43.29659,44.72984,3.401513,NaN,None,http://www.langerresearch.com/wp-content/uploa...,47881,74936,11/1/16,15:09:38 1 Nov 2016
2,2,2016,President,polls-plus,Clinton vs. Trump vs. Johnson,11/1/16,Virginia,10/27/2016,10/30/2016,ABC News/Washington Post,...,46.29779,40.72604,6.401513,NaN,None,https://www.washingtonpost.com/local/virginia-...,47880,74934,11/1/16,15:09:38 1 Nov 2016
3,3,2016,President,polls-plus,Clinton vs. Trump vs. Johnson,11/1/16,Florida,10/20/2016,10/24/2016,SurveyUSA,...,46.35931,45.30585,1.777730,NaN,None,http://www.baynews9.com/content/news/baynews9/...,47465,74252,10/25/16,15:09:38 1 Nov 2016
4,4,2016,President,polls-plus,Clinton vs. Trump vs. Johnson,11/1/16,U.S.,10/20/2016,10/25/2016,Pew Research Center,...,45.32744,42.20888,3.618320,NaN,None,http://www.people-press.org/2016/10/27/as-elec...,47616,74519,10/27/16,15:09:38 1 Nov 2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10231,10231,2016,President,polls-only,Clinton vs. Trump vs. Johnson,11/1/16,Alabama,9/30/2016,10/13/2016,Ipsos,...,37.30964,54.76821,NaN,NaN,None,http://reuters.com/statesofthenation/,46817,73263,10/15/16,14:57:58 1 Nov 2016
10232,10232,2016,President,polls-only,Clinton vs. Trump vs. Johnson,11/1/16,Virginia,9/30/2016,10/6/2016,Ipsos,...,49.13094,39.41588,NaN,NaN,None,http://www.reuters.com/statesofthenation/,46675,72969,10/10/16,14:57:58 1 Nov 2016
10233,10233,2016,President,polls-only,Clinton vs. Trump vs. Johnson,11/1/16,Virginia,9/16/2016,9/22/2016,Ipsos,...,45.97130,39.97518,NaN,NaN,None,http://www.reuters.com/statesofthenation/,46096,72088,9/26/16,14:57:58 1 Nov 2016
10234,10234,2016,President,polls-only,Clinton vs. Trump vs. Johnson,11/1/16,North Carolina,6/20/2016,6/21/2016,Public Policy Polling,...,45.29390,46.66175,1.596946,NaN,None,http://www.publicpolicypolling.com/pdf/2015/PP...,44400,67363,6/23/16,14:57:58 1 Nov 2016


In [4]:
clinton.columns

Index(['index', 'cycle', 'branch', 'type', 'matchup', 'forecastdate', 'state',
       'startdate', 'enddate', 'pollster', 'grade', 'samplesize', 'population',
       'poll_wt', 'rawpoll_clinton', 'rawpoll_trump', 'rawpoll_johnson',
       'rawpoll_mcmullin', 'adjpoll_clinton', 'adjpoll_trump',
       'adjpoll_johnson', 'adjpoll_mcmullin', 'multiversions', 'url',
       'poll_id', 'question_id', 'createddate', 'timestamp'],
      dtype='object')

In [5]:
tc_polls = pd.read_sql('''SELECT forecastdate, state, pollster, grade, samplesize, population, rawpoll_clinton, rawpoll_trump, 
                          rawpoll_clinton - rawpoll_trump as raw_differential,
                          adjpoll_clinton, adjpoll_trump, adjpoll_clinton - adjpoll_trump as adj_differential,
                          CASE WHEN (rawpoll_clinton - rawpoll_trump) > 0 THEN 'Clinton' ELSE 'Trump' END as Winner
                          FROM fivethirtyeight
                          ORDER BY forecastdate DESC, state, pollster''',conn)
tc_polls

,forecastdate,state,pollster,grade,samplesize,population,rawpoll_clinton,rawpoll_trump,raw_differential,adjpoll_clinton,adjpoll_trump,adj_differential,Winner
0,11/1/16,Alabama,Google Consumer Surveys,B,443.0,lv,20.38,56.12,-35.74,25.34553,61.86457,-36.51904,Trump
1,11/1/16,Alabama,Google Consumer Surveys,B,390.0,lv,22.36,56.83,-34.47,27.15753,63.75026,-36.59273,Trump
2,11/1/16,Alabama,Google Consumer Surveys,B,445.0,lv,23.68,53.37,-29.69,28.59634,60.81203,-32.21569,Trump
3,11/1/16,Alabama,Google Consumer Surveys,B,373.0,lv,19.61,54.78,-35.17,24.73898,62.43942,-37.70044,Trump
4,11/1/16,Alabama,Google Consumer Surveys,B,379.0,lv,22.51,53.80,-31.29,27.98772,61.46837,-33.48065,Trump
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10231,11/1/16,Wyoming,SurveyMonkey,C-,727.0,rv,21.00,57.00,-36.00,23.25227,61.62851,-38.37624,Trump
10232,11/1/16,Wyoming,SurveyMonkey,C-,361.0,lv,25.00,59.00,-34.00,23.39463,59.91796,-36.52333,Trump
10233,11/1/16,Wyoming,University of Wyoming,None,722.0,lv,20.00,58.00,-38.00,20.72080,59.93697,-39.21617,Trump
10234,11/1/16,Wyoming,University of Wyoming,None,722.0,lv,20.00,58.00,-38.00,20.61174,60.20643,-39.59469,Trump


In [6]:
tc_polls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10236 entries, 0 to 10235
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   forecastdate      10236 non-null  object 
 1   state             10236 non-null  object 
 2   pollster          10236 non-null  object 
 3   grade             9102 non-null   object 
 4   samplesize        10233 non-null  float64
 5   population        10236 non-null  object 
 6   rawpoll_clinton   10236 non-null  float64
 7   rawpoll_trump     10236 non-null  float64
 8   raw_differential  10236 non-null  float64
 9   adjpoll_clinton   10236 non-null  float64
 10  adjpoll_trump     10236 non-null  float64
 11  adj_differential  10236 non-null  float64
 12  Winner            10236 non-null  object 
dtypes: float64(7), object(6)
memory usage: 1.0+ MB


In [7]:
biden = pd.read_sql('''select * from trumpbiden''',conn)
biden

,index,question_id,poll_id,cycle,state,pollster_id,pollster,sponsor_ids,sponsors,display_name,...,created_at,notes,url,stage,race_id,answer,candidate_id,candidate_name,candidate_party,pct
0,0,136283,72621,2020,Iowa,383,Public Policy Polling,None,None,Public Policy Polling,...,11/2/20 09:02,None,https://www.publicpolicypolling.com/wp-content...,general,6223,Biden,13256,Joseph R. Biden Jr.,DEM,49.0
1,1,136283,72621,2020,Iowa,383,Public Policy Polling,None,None,Public Policy Polling,...,11/2/20 09:02,None,https://www.publicpolicypolling.com/wp-content...,general,6223,Trump,13254,Donald Trump,REP,48.0
2,2,136322,72647,2020,Pennsylvania,461,Susquehanna Polling & Research Inc.,None,None,Susquehanna Polling & Research Inc.,...,11/2/20 12:49,None,https://www.realclearpolitics.com/docs/2020/Su...,general,6249,Biden,13256,Joseph R. Biden Jr.,DEM,48.4
3,3,136322,72647,2020,Pennsylvania,461,Susquehanna Polling & Research Inc.,None,None,Susquehanna Polling & Research Inc.,...,11/2/20 12:49,None,https://www.realclearpolitics.com/docs/2020/Su...,general,6249,Trump,13254,Donald Trump,REP,49.2
4,4,136322,72647,2020,Pennsylvania,461,Susquehanna Polling & Research Inc.,None,None,Susquehanna Polling & Research Inc.,...,11/2/20 12:49,None,https://www.realclearpolitics.com/docs/2020/Su...,general,6249,Jorgensen,14611,Jo Jorgensen,LIB,1.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16433,16433,92081,57026,2020,Ohio,383,Public Policy Polling,1363.0,DEM (partisan),Public Policy Polling,...,12/6/18 14:45,None,https://www.cleveland.com/politics/2018/11/thi...,general,6246,Trump,13254,Donald Trump,REP,46.0
16434,16434,92082,57026,2020,Ohio,383,Public Policy Polling,1363.0,DEM (partisan),Public Policy Polling,...,12/6/18 14:46,None,https://www.cleveland.com/politics/2018/11/thi...,general,6246,Warren,13258,Elizabeth Warren,DEM,43.0
16435,16435,92082,57026,2020,Ohio,383,Public Policy Polling,1363.0,DEM (partisan),Public Policy Polling,...,12/6/18 14:46,None,https://www.cleveland.com/politics/2018/11/thi...,general,6246,Trump,13254,Donald Trump,REP,49.0
16436,16436,92078,57025,2020,None,399,Rasmussen Reports/Pulse Opinion Research,None,None,Rasmussen Reports/Pulse Opinion Research,...,12/6/18 14:40,None,http://www.rasmussenreports.com/public_content...,general,6210,Obama,13253,Michelle Obama,DEM,50.0


In [8]:
biden.columns

Index(['index', 'question_id', 'poll_id', 'cycle', 'state', 'pollster_id',
       'pollster', 'sponsor_ids', 'sponsors', 'display_name',
       'pollster_rating_id', 'pollster_rating_name', 'fte_grade',
       'sample_size', 'population', 'population_full', 'methodology',
       'office_type', 'seat_number', 'seat_name', 'start_date', 'end_date',
       'election_date', 'sponsor_candidate', 'internal', 'partisan',
       'tracking', 'nationwide_batch', 'ranked_choice_reallocated',
       'created_at', 'notes', 'url', 'stage', 'race_id', 'answer',
       'candidate_id', 'candidate_name', 'candidate_party', 'pct'],
      dtype='object')

In [9]:
biden_polls = pd.read_sql('''SELECT cycle, state, pollster, sponsors, display_name, fte_grade, sample_size, population, population_full,
                             methodology, office_type, end_date, sponsor_candidate, internal, partisan, notes, candidate_name, pct
                          FROM trumpbiden
                          WHERE candidate_name like '%Trump' OR candidate_name like '%Biden Jr.'
                          ORDER BY end_date, state, pollster''',conn)
biden_polls

,cycle,state,pollster,sponsors,display_name,fte_grade,sample_size,population,population_full,methodology,office_type,end_date,sponsor_candidate,internal,partisan,notes,candidate_name,pct
0,2020,Texas,Neighborhood Research and Media,Courageous Conservatives PAC,Atlantic Media and Research,B/C,504.0,lv,lv,Live Phone,U.S. President,1/11/19,None,0,REP,None,Donald Trump,52.00
1,2020,None,IBD/TIPP,None,IBD/TIPP,A/B,846.0,rv,rv,Live Phone,U.S. President,1/11/20,None,0,None,None,Joseph R. Biden Jr.,48.00
2,2020,None,IBD/TIPP,None,IBD/TIPP,A/B,846.0,rv,rv,Live Phone,U.S. President,1/11/20,None,0,None,None,Donald Trump,46.00
3,2020,None,IBD/TIPP,None,IBD/TIPP,A/B,846.0,rv,rv,Live Phone,U.S. President,1/11/20,None,0,None,None,Donald Trump,47.00
4,2020,None,IBD/TIPP,None,IBD/TIPP,A/B,846.0,rv,rv,Live Phone,U.S. President,1/11/20,None,0,None,None,Donald Trump,47.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13586,2020,None,USC Dornsife/Los Angeles Times,None,USC Dornsife,B/C,5241.0,lv,lv,Online,U.S. President,9/9/20,None,0,None,traditional voting question,Donald Trump,40.97
13587,2020,None,USC Dornsife/Los Angeles Times,None,USC Dornsife,B/C,5116.0,rv,rv,Online,U.S. President,9/9/20,None,0,None,traditional voting question (RV),Joseph R. Biden Jr.,51.05
13588,2020,None,USC Dornsife/Los Angeles Times,None,USC Dornsife,B/C,5116.0,rv,rv,Online,U.S. President,9/9/20,None,0,None,traditional voting question (RV),Donald Trump,40.72
13589,2020,Wisconsin,Morning Consult,None,Morning Consult,B/C,816.0,lv,lv,Online,U.S. President,9/9/20,None,0,None,None,Joseph R. Biden Jr.,50.00


In [10]:
biden_polls['methodology'].unique()

array(['Live Phone', 'Online', 'IVR/Online', 'Live Phone/Online',
       'Online/IVR', 'Online/Text', 'IVR/Text', None, 'Automated Phone',
       'IVR/Online/Text', 'IVR/Live Phone', 'Text', 'Mail',
       'Live Phone/Text', 'Live Phone/Online/Text',
       'IVR/Live Phone/Online'], dtype=object)

In [11]:
biden_polls['fte_grade'].unique()

array(['B/C', 'A/B', 'B+', 'C+', 'B', 'A', 'B-', None, 'A-', 'A+', 'C',
       'D-', 'C/D', 'C-'], dtype=object)

In [12]:
biden_polls["state"] = biden_polls.state.fillna("U.S.")
biden_polls

,cycle,state,pollster,sponsors,display_name,fte_grade,sample_size,population,population_full,methodology,office_type,end_date,sponsor_candidate,internal,partisan,notes,candidate_name,pct
0,2020,Texas,Neighborhood Research and Media,Courageous Conservatives PAC,Atlantic Media and Research,B/C,504.0,lv,lv,Live Phone,U.S. President,1/11/19,None,0,REP,None,Donald Trump,52.00
1,2020,U.S.,IBD/TIPP,None,IBD/TIPP,A/B,846.0,rv,rv,Live Phone,U.S. President,1/11/20,None,0,None,None,Joseph R. Biden Jr.,48.00
2,2020,U.S.,IBD/TIPP,None,IBD/TIPP,A/B,846.0,rv,rv,Live Phone,U.S. President,1/11/20,None,0,None,None,Donald Trump,46.00
3,2020,U.S.,IBD/TIPP,None,IBD/TIPP,A/B,846.0,rv,rv,Live Phone,U.S. President,1/11/20,None,0,None,None,Donald Trump,47.00
4,2020,U.S.,IBD/TIPP,None,IBD/TIPP,A/B,846.0,rv,rv,Live Phone,U.S. President,1/11/20,None,0,None,None,Donald Trump,47.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13586,2020,U.S.,USC Dornsife/Los Angeles Times,None,USC Dornsife,B/C,5241.0,lv,lv,Online,U.S. President,9/9/20,None,0,None,traditional voting question,Donald Trump,40.97
13587,2020,U.S.,USC Dornsife/Los Angeles Times,None,USC Dornsife,B/C,5116.0,rv,rv,Online,U.S. President,9/9/20,None,0,None,traditional voting question (RV),Joseph R. Biden Jr.,51.05
13588,2020,U.S.,USC Dornsife/Los Angeles Times,None,USC Dornsife,B/C,5116.0,rv,rv,Online,U.S. President,9/9/20,None,0,None,traditional voting question (RV),Donald Trump,40.72
13589,2020,Wisconsin,Morning Consult,None,Morning Consult,B/C,816.0,lv,lv,Online,U.S. President,9/9/20,None,0,None,None,Joseph R. Biden Jr.,50.00
